<a href="https://colab.research.google.com/github/apspatz/TheMonkeyCage/blob/master/The_Monkey_Cage_v02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Given a sequence of characters from this data (e.g. "Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly. 

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


This code borrows strongly from TensorFlow's tutorial on text generation using an RNN. For more information please visit TensorFlow's website directly:

https://www.tensorflow.org/tutorials/text/text_generation


Setup:

In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

TensorFlow 2.x selected.


Paths to the Homer text file (which contains both the Odyssey & the Illiad):

In [9]:
path_to_file_homer = tf.keras.utils.get_file('HomerComplete_edited.txt', 'https://mandrewsbucket.s3.amazonaws.com/HomerComplete_edited.txt')

1458176/1456151 [==============================] - 0s 0us/step


Path to the Shakespeare text file:

In [12]:
path_to_file_shakespeare = tf.keras.utils.get_file('shakespeare_edited.txt', 'https://mandrewsbucket.s3.amazonaws.com/shakespeare_edited.txt')

5464064/5458199 [==============================] - 2s 0us/step


Path to the Bible text file:

In [9]:
path_to_file_bible = tf.keras.utils.get_file('bible_edited.txt', 'https://mandrewsbucket.s3.amazonaws.com/bible_edited.txt')

4366336/4365973 [==============================] - 1s 0us/step


Path to the Sherlock text file:

In [10]:
path_to_file_sherlock = tf.keras.utils.get_file('sherlock_edited.txt', 'https://mandrewsbucket.s3.amazonaws.com/sherlock_edited.txt')

614400/609377 [==============================] - 1s 2us/step


Path to the Dante text files (containing The Divine Comedy, Paradise, The Divine Comedy, Purgatory, and The Vision of Hell):

In [14]:
path_to_file_dante = tf.keras.utils.get_file('DivineComedyComplete_edited.txt', 'https://mandrewsbucket.s3.amazonaws.com/DivineComedyComplete_edited.txt')

622592/622204 [==============================] - 1s 1us/step


In [12]:
# Read, then decode for py2 compat.
text = open(path_to_file_homer, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of Homer text: {} characters'.format(len(text)))

Length of Homer text: 1445041 characters


In [13]:
# Take a look at the first 250 characters in text
print(text[:250])

BOOK I

Sing, O goddess, the anger of Achilles son of Peleus, that brought
countless ills upon the Achaeans. Many a brave soul did it send hurrying
down to Hades, and many a hero did it yield a prey to dogs and vultures,
for so were the counsels of J


In [14]:
print(text[-550:])

o earth, and some confusedly fly.
With dreadful shouts Ulysses pour’d along,
Swift as an eagle, as an eagle strong.
But Jove’s red arm the burning thunder aims:
Before Minerva shot the livid flames;
Blazing they fell, and at her feet expired;
Then stopped the goddess, trembled and retired.

“Descended from the gods! Ulysses, cease;
Offend not Jove: obey, and give the peace.”

So Pallas spoke: the mandate from above
The king obey’d. The virgin-seed of Jove,
In Mentor’s form, confirm’d the full accord,
And willing nations knew their lawful lord.



In [0]:
# text = text[251:-555]
# print(text[0:250])
# print(text[-250:])

In [16]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

72 unique characters


### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [18]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

B
O
O
K
 


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [23]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'BOOK I\n\nSing, O goddess, the anger of Achilles son of Peleus, that brought\ncountless ills upon the A'
Target data: 'OOK I\n\nSing, O goddess, the anger of Achilles son of Peleus, that brought\ncountless ills upon the Ac'


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "B" and trys to predict the index for "O" as the next character (in the case of using Homer text file). At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [24]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 15 ('B')
  expected output: 28 ('O')
Step    1
  input: 28 ('O')
  expected output: 28 ('O')
Step    2
  input: 28 ('O')
  expected output: 24 ('K')
Step    3
  input: 24 ('K')
  expected output: 1 (' ')
Step    4
  input: 1 (' ')
  expected output: 22 ('I')


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [25]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [29]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 72) # (batch_size, sequence_length, vocab_size)


In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           18432     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 72)            73800     
Total params: 4,030,536
Trainable params: 4,030,536
Non-trainable params: 0
_________________________________________________________________


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [31]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 72)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.277217


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [0]:
model.compile(optimizer='adam', loss=loss)

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the Training

In Colab, set the runtime to GPU for faster training.

In [0]:
EPOCHS=40

In [35]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 223 steps
Epoch 1/40
223/223 [==============================] - 15s 66ms/step - loss: 2.4805
Epoch 2/40
223/223 [==============================] - 14s 62ms/step - loss: 1.8315
Epoch 3/40
223/223 [==============================] - 14s 63ms/step - loss: 1.5722
Epoch 4/40
223/223 [==============================] - 14s 65ms/step - loss: 1.4418
Epoch 5/40
223/223 [==============================] - 15s 67ms/step - loss: 1.3643
Epoch 6/40
223/223 [==============================] - 15s 67ms/step - loss: 1.3081
Epoch 7/40
223/223 [==============================] - 15s 69ms/step - loss: 1.2621
Epoch 8/40
223/223 [==============================] - 15s 69ms/step - loss: 1.2214
Epoch 9/40
223/223 [==============================] - 16s 70ms/step - loss: 1.1840
Epoch 10/40
223/223 [==============================] - 15s 68ms/step - loss: 1.1478
Epoch 11/40
223/223 [==============================] - 15s 68ms/step - loss: 1.1131
Epoch 12/40
223/223 [==============================] - 15s 67ms/s

## Generate Text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [36]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_40'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [38]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            18432     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 72)             73800     
Total params: 4,030,536
Trainable params: 4,030,536
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [40]:
print(generate_text(model, start_string=u"The sea "))

The sea spear, I came, but some neward the retireoid
the ways of way,
And each be fill’d on every side from out about
the wall. It was yet at escape from which he strong far others to display’d Hector; he prowe
So year, a vase supplies,
On wove, Saschet: the hero ran,
White swains the vestur send to Agamemnon; but the
young men away, and that you were struck
with the obscures of Mt. Placus to the host of kind);
Whose boughs secused full of gone, the prince with state arphichos
strongly rush in wrantly on his body. Hector
was nearolong up all was both to chase at accountely, by the stern of
the son of Atreus coming through the gooder gening by the roads, at once,
and your sire crossed the chariot. Menelaus then tole
him with embattled, "which of the Lycianae,
warrior- to easly into the finger of good in fumb, and I will gain noble kinddy hold his anger.
It went to Troy as I can. The snatcher-banquety of battle with wind-carefully on the plain
for battle? we will yet fell sprawling upon 